有三种计算图的构建方式：`静态计算图`，`动态计算图`，以及 `AutoGraph`

* 静态计算图：静态计算则意味着程序在编译执行时将先生成神经网络的结构，然后再执行相应操作。从理论上讲，静态计算这样的机制允许编译器进行更大程度的优化，但是这也意味着你所期望的程序与编译器实际执行之间存在着更多的代沟。这也意味着，代码中的错误将更加难以发现（比如，如果计算图的结构出现问题，你可能只有在代码执行到相应操作的时候才能发现它)

* 动态计算图：动态计算意味着程序将按照我们编写命令的顺序进行执行。这种机制将使得调试更加容易，并且也使得我们将大脑中的想法转化为实际代码变得更加容易。

`TensorFlow 2.0`主要使用的是动态计算图和Autograph。
而`Autograph机制`可以将动态图转换成静态计算图，兼收执行效率和编码效率之利。

动态计算图易于调试，编码效率较高，但执行效率偏低。

静态计算图执行效率很高，但较难调试。

`AutoGraph`在`TensorFlow 2.0`通过`@tf.function`实现的。





## Autograph使用规范

1、被`@tf.function`修饰的函数应尽量使用`TensorFlow`中的函数而不是`Python`中的其他函数。

In [3]:
import numpy as np
import tensorflow as tf

@tf.function
def np_random():
    a = np.random.randn(3,3)
    tf.print(a)

@tf.function
def tf_random():
    a = tf.random.normal((3,3))
    tf.print(a)

In [4]:
# np_random每次执行都是一样的结果。
np_random()
np_random()

array([[-0.31820445,  0.64910653, -0.90965468],
       [-0.00367398, -0.56696726, -0.82609263],
       [-0.3193194 , -0.32913403, -0.02126509]])
array([[-0.31820445,  0.64910653, -0.90965468],
       [-0.00367398, -0.56696726, -0.82609263],
       [-0.3193194 , -0.32913403, -0.02126509]])


In [9]:
np.random.randn(3,3)

array([[ 0.91045967, -0.353947  ,  0.54314763],
       [ 0.54683283,  1.04354545,  0.57993926],
       [-1.36315617, -1.24973397, -1.69570283]])

In [6]:
np.random.randn(3,3)

array([[ 0.05022144, -0.0376609 , -1.23237842],
       [ 0.44842382, -0.91374959, -0.1011143 ],
       [ 1.23738648, -0.4243661 ,  0.20236161]])

In [7]:
tf_random()
tf_random()

[[0.536422312 -0.269435495 -1.0883702]
 [0.413602948 0.416455567 -0.610887229]
 [1.26911819 1.98646414 1.05896056]]
[[-0.0671163574 -0.394631922 0.177696884]
 [0.771233857 -1.61399925 -0.011960783]
 [-0.0439407714 -0.182855 0.555179954]]


2、避免在`@tf.function`修饰的函数内部定义`tf.Variable`.

In [11]:

x = tf.Variable(1.0, dtype=tf.float32)

@tf.function
def outer_var():
    x.assign_add(1.0)
    tf.print(x)
    return(x)

outer_var() 
outer_var()

2
3


<tf.Tensor: shape=(), dtype=float32, numpy=3.0>

In [12]:
# 报错
@tf.function
def inner_var():
    x = tf.Variable(1.0, dtype = tf.float32)
    x.assign_add(1.0)
    tf.print(x)
    return(x)
inner_var()

ValueError: in user code:

    <ipython-input-12-62549d1412ca>:4 inner_var  *
        x = tf.Variable(1.0, dtype = tf.float32)
    /home/zhangchunmei/anaconda3/envs/TF2/lib/python3.7/site-packages/tensorflow/python/ops/variables.py:262 __call__  **
        return cls._variable_v2_call(*args, **kwargs)
    /home/zhangchunmei/anaconda3/envs/TF2/lib/python3.7/site-packages/tensorflow/python/ops/variables.py:256 _variable_v2_call
        shape=shape)
    /home/zhangchunmei/anaconda3/envs/TF2/lib/python3.7/site-packages/tensorflow/python/ops/variables.py:67 getter
        return captured_getter(captured_previous, **kwargs)
    /home/zhangchunmei/anaconda3/envs/TF2/lib/python3.7/site-packages/tensorflow/python/eager/def_function.py:702 invalid_creator_scope
        "tf.function-decorated function tried to create "

    ValueError: tf.function-decorated function tried to create variables on non-first call.


3、被`@tf.function`修饰的函数不可修改该函数外部的`Python`列表或字典等结构类型变量。

In [13]:
tensor_list = []

# 不加入该修饰器时，列表元素可以改变
# @tf.function
def append_tensor(x):
    tensor_list.append(x)
    return tensor_list

append_tensor(tf.constant(5.0))
append_tensor(tf.constant(6.0))
print(tensor_list)

[<tf.Tensor: shape=(), dtype=float32, numpy=5.0>, <tf.Tensor: shape=(), dtype=float32, numpy=6.0>]


In [14]:
tensor_list = []

# 加上这一行切换成Autograph结果将不符合预期！！！无法修改列表元素
@tf.function 
def append_tensor(x):
    tensor_list.append(x)
    return tensor_list


append_tensor(tf.constant(5.0))
append_tensor(tf.constant(6.0))
print(tensor_list)

[<tf.Tensor 'x:0' shape=() dtype=float32>]
